<a href="https://colab.research.google.com/github/kynthesis/HaystackResearch/blob/main/8_Conversational_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cách xây dựng một chatbot sử dụng Agent**



# 1. Kiểm tra GPU runtime

In [1]:
%%bash

nvidia-smi

Sun Jul  2 12:54:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 2. Cài đặt Haystack

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]

# 3. Bật chế độ logging

In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

# 4. Sử dụng API HuggingFace

In [4]:
from getpass import getpass

model_api_key = getpass("Enter model provider API key:")

Enter model provider API key:··········


# 5. Khởi tạo Prompt

In [5]:
from haystack.nodes import PromptNode

model_name = "OpenAssistant/oasst-sft-1-pythia-12b"
prompt_node = PromptNode(model_name, api_key=model_api_key, max_length=256)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


# 6. Khởi tạo Memory

In [6]:
from haystack.agents.memory import ConversationSummaryMemory

summary_memory = ConversationSummaryMemory(prompt_node)

/usr/local/lib/python3.10/dist-packages/haystack/nodes/prompt/prompt_template.py:444: UserWarning: You're using a legacy prompt template 'conversational-summary', we strongly suggest you use prompts from the official Haystack PromptHub: https://prompthub.deepset.ai/
  warnings.warn(


# 7. Khởi tạo Agent

In [8]:
from haystack.agents.conversational import ConversationalAgent

conversational_agent = ConversationalAgent(prompt_node=prompt_node, memory=summary_memory)

/usr/local/lib/python3.10/dist-packages/haystack/nodes/prompt/prompt_template.py:444: UserWarning: You're using a legacy prompt template 'conversational-agent-without-tools', we strongly suggest you use prompts from the official Haystack PromptHub: https://prompthub.deepset.ai/
  warnings.warn(


# 8. Thử nghiệm Chatbot

In [9]:
import ipywidgets as widgets
from IPython.display import clear_output

## Text Input
user_input = widgets.Textarea(
    value="",
    placeholder="Type your prompt here",
    disabled=False,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="100%", height="100%"),
)

## Submit Button
submit_button = widgets.Button(
    description="Submit", button_style="success", layout=widgets.Layout(width="100%", height="80%")
)


def on_button_clicked(b):
    user_prompt = user_input.value
    user_input.value = ""
    print("\nUser:\n", user_prompt)
    conversational_agent.run(user_prompt)


submit_button.on_click(on_button_clicked)

## Show Memory Button
memory_button = widgets.Button(
    description="Show Memory", button_style="info", layout=widgets.Layout(width="100%", height="100%")
)


def on_memory_button_clicked(b):
    memory = conversational_agent.memory.load()
    if len(memory):
        print("\nMemory:\n", memory)
    else:
        print("Memory is empty")


memory_button.on_click(on_memory_button_clicked)

## Clear Memory Button
clear_button = widgets.Button(
    description="Clear Memory", button_style="warning", layout=widgets.Layout(width="100%", height="100%")
)


def on_clear_button_button_clicked(b):
    conversational_agent.memory.clear()
    print("\nMemory is cleared\n")


clear_button.on_click(on_clear_button_button_clicked)

## Layout
grid = widgets.GridspecLayout(3, 3, height="200px", width="800px", grid_gap="10px")
grid[0, 2] = clear_button
grid[0:2, 0:2] = user_input
grid[2, 0:] = submit_button
grid[1, 2] = memory_button
display(grid)

GridspecLayout(children=(Button(button_style='warning', description='Clear Memory', layout=Layout(grid_area='w…


User:
 What is the biggest city in Vietnam

Agent conversational-agent-without-tools started with {'query': 'What is the biggest city in Vietnam', 'params': None}
Hanoi
User:
 Tell me something about that city

Agent conversational-agent-without-tools started with {'query': 'Tell me something about that city', 'params': None}
Hanoi is the capital and largest city of Vietnam. It is located in the northern part of the country, on the banks of the Red River. The city has a rich history, dating back to the 12th century when it was known as Hanoi. It was the seat of the Nguyen Dynasty, which ruled over Vietnam from 1802 to 1945. During this time, Hanoi became an important cultural and political center, and it remains a major hub of commerce and industry in Vietnam today.

Hanoi is known for its rich architecture, including the Old Quarter, which is a maze of narrow streets and historic buildings. The city is also home to many museums, galleries, and cultural institutions, which offer a uni